# Association Rules - Life in L.A. <a name=top></a>

In this notebook, you will apply association rules mining to the `Life in L.A.` dataset.

The dataset consists of 16 observations, each item having 8 variables. Our goal here is to find the combination of items that occur with certain frequency, and find item-sets that provide insights to the structure of data

#### TUTORIAL OUTLINE

1. [Load and Explore Data](#load)
2. [Explore Data With `arules`](#arules)
3. [Visualizing Association Rules](#visual)
4. [Exercise](#exer)
4. [Other References](#others)

---
[Back to top](#top)
## 1. LOAD AND EXPLORE DATA <a name=load></a>

The very fisrt step in programming with R is to import data. Then we perform a few routines to explore the structure of data.

In [0]:
#life.in.LA <- read.csv("../Data/turner_life_in_LA.csv", head=TRUE)
life.in.LA <- read.csv("Data/turner_life_in_LA.csv", head=TRUE)

head(life.in.LA) # look at data (first 6 rows)
str(life.in.LA) # investigate types of variables; other than lon & lat, features seem to be "categorical"
summary(life.in.LA) # provides 6-point summary for each variable

---
[Back to top](#top)
## 2. EXPLORING DATA WITH `arules`<a name=arules></a>
In previous section, we learned that variables `affluence`, `unemployment`, `urban.stress`, and `prop.white`, are technically categorical variables. We will convert these integer values to categories by function called `lapply`. Then we use a function called `apriori`, which returns the all possible rules (itemset).

By default, `apriori` creates rules with minimum support of $0.1$, minimum confidence of $0.8$, and maximum of $10$ items.

**NOTE:** since `apriori` is a function within a package called `arules`, we must load the library beforehand. 

In [0]:
library(arules)

(life.in.LA[,c(3,4,5,6)] <- lapply(life.in.LA[,c(3,4,5,6)] , factor)) # Categorizes variables 3 to 6

# find association rules with default settings
rules.LA <- apriori(life.in.LA[,3:8])

For a rule $X \to Y$, `arules` computes the following metrics: 
 - $\text{Support}(X\to Y) = \frac{\text{Freq}(X,Y)}{N}$
 - $\text{Confidence}(X\to Y) = \frac{\text{Freq}(X,Y)}{\text{Freq}(X)}$
 - $\text{Lift}(X\to Y) = \frac{N^2 \times \text{Support}(X\to Y)}{\text{Freq}(X) \times \text{Freq}(Y)}$ 

In some instances, the higher these values, the better. In other instances, we might be interested in finding rules for which the values could be quite small. 

---
The following function `inspect` returns a list of all rules created by `apriori`. Using the default setting, the list consists of $444$ rules (which is quite lengthy... it looks like the default setting is very lenient). 

In [0]:
inspect(rules.LA)

---
Now, the fun part comes with setting our own parameters to create the list of item-sets! The new object `rules2.LA` now consists of item-sets that have a minimum support of $0.5$, and minimum confidence of $0.5$.

In [0]:
rules2.LA <- apriori(life.in.LA[,3:8],parameter = list(supp=0.50, conf=0.5),control = list(verbose=F))
rules2.LA.sorted <- sort(rules2.LA, by=c("lift"), decreasing=TRUE) # sort the list by lift (in descending order)
inspect(rules2.LA.sorted) # print out the results

---
We have a list of 7 rules in above, but are they all **independent** rules?<br\><br\>
The answer is $\textbf{NO!}$ For example, we have rules `{prop.white=1} => {unemployment=2}`, and `{unemployment=2} => {prop.white=1}`, which are simply reciprocals of each other. Hence the rule `{unemployment=2} => {prop.white=1}` is redundant.<br\><br\>

Which of the 2 rules should we remove? `{prop.white=1} => {unemployment=2}`, or {unemployment=2} => {prop.white=1}? <br\>

This is not an easy question to answer. To make a point, consider two situations

- Example 1: We asked people about their favourite sport and a team/player. We found that $100\%$ of people who said that the Ottawa Senators were their favourite team  gave hockey as their favourite sport. We also found that $40\%$ of hockey lovers said they cheer for Sens.

- Example 2: Last year, $40\%$ of successful candidates for a company were graduate students, and $100\%$ of graduate students who applied to this company got a job offer.

In these scenarios, which statements are more interesting?

In what follows, we attempt to remove all redundant rules. As a result, we remove $2$ rules.

In [0]:
subset.matrix <- as.matrix(is.subset(rules2.LA.sorted, rules2.LA.sorted))
subset.matrix[lower.tri(subset.matrix, diag=T)] <- NA
subset.matrix
(redundant.LA <- colSums(subset.matrix, na.rm=T) >= 1)
which(redundant.LA)

# remove redundant rules
rules2.LA.pruned <- rules2.LA.sorted[!redundant.LA]
inspect(rules2.LA.pruned)


---
[Back to top](#top)
## 3. VISUALIZING ASSOCIATION RULES <a name=visual></a>

So far, we sought interesting associations between attributes, and they were presented in tabular format. Now, let's look at the graphical representation of the association rule (by using package `arulesViz`). Will we able to extract information?

Unfortunately, the graphical representations do not bring much insight with the parameters given above. The first plot is very, very messy as we have $444$ item-sets to graph. The second graph has much clearner looking; empty-sets direct to  $3$ single item-sets (`urban.stress=1`, `unemployment=2`, and `prop.white=1`). We also see loops among `urban.stress`, `unemployment`, and `prop.white`. These loops correspond to what we considered to be redundant. The third plot is even simpler; the loops are removed from the plot.


In [0]:
library(arulesViz)

plot(rules.LA, method="graph", control=list(type="items")) # This one is super messy
plot(rules2.LA, method="graph", control=list(type="items")) # This one is simple
plot(rules2.LA.pruned, method="graph", control=list(type="items")) # This is even simpler (too simple?)


---
[Back to top](#top)
## 4. EXERCISES<a name=exer></a>
- repeat this process for the `Transactions.csv` dataset.
- repeat this process for the `titanic` dataset, which is defined below. 

In [0]:
# load and explore data
class=c(rep("3rd",52),rep("1st",118),rep("2nd",154),rep("3rd",387),rep("Crew",670),rep("1st",4),rep("2nd",13.01),rep("3rd",89),rep("Crew",3),rep("1st",5),rep("2nd",11),rep("3rd",13),rep("1st",1),rep("2nd",13),rep("3rd",14),rep("1st",57),rep("2nd",14),rep("3rd",75),rep("Crew",192),rep("1st",140),rep("2nd",80),rep("3rd",76),rep("Crew",20))
sex=c(rep("Male",35),rep("Female",17),rep("Male",1329),rep("Female",109),rep("Male",29),rep("Female",28),rep("Male",338),rep("Female",316))
age=c(rep("Child",52),rep("Adult",1438),rep("Child",57),rep("Adult",654))
survived=c(rep("No",1490),rep("Yes",711))

titanic=data.frame(class,sex,age,survived)

str(titanic)
summary(titanic)
table(titanic$age,titanic$survived)
table(titanic$class,titanic$survived)


---
- Look for rules where the consequent consists of either "Survived" levels (i.e., we're trying to figure out what lead to either survival or lack of survival). This can be arranged by adding the `appearance` parameters to the apriori command, as below:  

`rules2 <- apriori(titanic,parameter = list(minlen=2, supp=0.005, conf=0.8),appearance = list(rhs=c("survived=No", "survived=Yes"),default="lhs"),control = list(verbose=F))`

- Set the parameters to reduce the number of rules: rules of size at least 2, with support at least greater than 0.005 and confidence greater than 0.8. 

---
[Back to top](#top)
## 5. OTHER REFERENCES <a name=others></a>
- http://www.saedsayad.com/association_rules.htm
- http://www.rdatamining.com/examples/association-rules
- https://cran.r-project.org/web/packages/arulesViz/vignettes/arulesViz.pdf
- http://www.lynda.com/R-tutorials/Up-Running-R/120612-2.html?
- http://michael.hahsler.net/research/arules_RUG_2015/demo/ (very good example)